In [ ]:
# 기본적인 부분
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc("font", family="NanumGothic")
plt.rcParams["axes.unicode_minus"]=False

# 데이터 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

# 학습 알고리즘
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet

from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error, mean_absolute_error
from sklearn.metrics import classification_report
from scipy.special import expit, softmax

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree

from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.proportion import proportions_ztest

from sklearn.ensemble import RandomForestClassifier

# 서울시 상권 월간 매출 예측 모델

## 상권 추정매출

In [ ]:
seoul_sales = pd.read_csv("../data/서울시 상권분석서비스(추정매출-상권).csv", encoding="cp949")
seoul_sales.shape

In [ ]:
seoul_sales.head()

In [ ]:
seoul_sales.info()

In [ ]:
seoul_sales_copy=seoul_sales.copy()

In [ ]:
seoul_sales_copy["출근_매출_금액"] = (seoul_sales_copy["시간대_00~06_매출_금액"] + seoul_sales_copy["시간대_06~11_매출_금액"])
seoul_sales_copy["점심_매출_금액"] = (seoul_sales_copy["시간대_11~14_매출_금액"] + seoul_sales_copy["시간대_14~17_매출_금액"])
seoul_sales_copy["퇴근_매출_금액"] = (seoul_sales_copy["시간대_17~21_매출_금액"] + seoul_sales_copy["시간대_21~24_매출_금액"])

columns_to_drop = [
    "서비스_업종_코드", "서비스_업종_코드_명", "상권_구분_코드", "상권_코드",
    "월요일_매출_금액", "화요일_매출_금액", "수요일_매출_금액", "목요일_매출_금액", "금요일_매출_금액",
    "토요일_매출_금액", "일요일_매출_금액",
    "시간대_00~06_매출_금액", "시간대_06~11_매출_금액",
    "시간대_11~14_매출_금액", "시간대_14~17_매출_금액",
    "시간대_17~21_매출_금액", "시간대_21~24_매출_금액",
    "연령대_10_매출_금액", "연령대_20_매출_금액", "연령대_30_매출_금액", 
    "연령대_40_매출_금액", "연령대_50_매출_금액", "연령대_60_이상_매출_금액", 
    "당월_매출_건수", "주중_매출_건수", "주말_매출_건수",
    "월요일_매출_건수", "화요일_매출_건수", "수요일_매출_건수", "목요일_매출_건수", "금요일_매출_건수",
    "토요일_매출_건수", "일요일_매출_건수",
    "시간대_건수~06_매출_건수", "시간대_건수~11_매출_건수",
    "시간대_건수~14_매출_건수", "시간대_건수~17_매출_건수",
    "시간대_건수~21_매출_건수", "시간대_건수~24_매출_건수",
    "남성_매출_건수", "여성_매출_건수",
    "연령대_10_매출_건수", "연령대_20_매출_건수", "연령대_30_매출_건수",
    "연령대_40_매출_건수", "연령대_50_매출_건수", "연령대_60_이상_매출_건수"
]

seoul_sales_copy = seoul_sales_copy.drop(columns=columns_to_drop)
seoul_sales_copy.info()

In [ ]:
columns_to_sum = seoul_sales_copy.columns[3:20]  # 매출 컬럼

agg_func = {}
# 기준_년분기_코드, 상권_구분_코드_명, 상권_코드_명
for col in seoul_sales_copy.columns[:3]:
    agg_func[col] = "first"
# 나머지
for col in columns_to_sum:
    agg_func[col] = "sum"

seoul_sales_copy_grouped = seoul_sales_copy.groupby(["상권_코드_명", "기준_년분기_코드"]).agg(agg_func)
seoul_sales_copy_grouped.head()

In [ ]:
seoul_sales_copy_grouped = seoul_sales_copy_grouped.reset_index(drop=True)

In [ ]:
seoul_sales_copy_grouped_sort = seoul_sales_copy_grouped.sort_values(by="기준_년분기_코드", ascending=True)

In [ ]:
seoul_sales_copy_grouped_sort.head()

In [ ]:
seoul_sales_copy_grouped_sort = seoul_sales_copy_grouped_sort.reset_index(drop=True)

In [ ]:
seoul_sales_copy_grouped_sort.head()

In [ ]:
seoul_sales_copy_grouped_sort["상권_코드_명"].unique()

## 유동인구

In [ ]:
seoul_people_street=pd.read_csv("../data/서울시 상권분석서비스(길단위인구-상권).csv", encoding="cp949")
seoul_people_street.info()

In [ ]:
seoul_people_street_copy=seoul_people_street.copy()

In [ ]:
seoul_people_street_copy["출근_유동인구_수"] = (seoul_people_street_copy["시간대_00_06_유동인구_수"] + seoul_people_street_copy["시간대_06_11_유동인구_수"])
seoul_people_street_copy["점심_유동인구_수"] = (seoul_people_street_copy["시간대_11_14_유동인구_수"] + seoul_people_street_copy["시간대_14_17_유동인구_수"])
seoul_people_street_copy["퇴근_유동인구_수"] = (seoul_people_street_copy["시간대_17_21_유동인구_수"] + seoul_people_street_copy["시간대_21_24_유동인구_수"])

seoul_people_street_copy["주중_유동인구_수"] = (seoul_people_street_copy["월요일_유동인구_수"] +
                                                 seoul_people_street_copy["화요일_유동인구_수"] +
                                                 seoul_people_street_copy["수요일_유동인구_수"] +
                                                 seoul_people_street_copy["목요일_유동인구_수"] +
                                                 seoul_people_street_copy["금요일_유동인구_수"])
seoul_people_street_copy["주말_유동인구_수"] = (seoul_people_street_copy["토요일_유동인구_수"] +
                                                 seoul_people_street_copy["일요일_유동인구_수"])

columns_to_drop = [
    "상권_구분_코드", "상권_코드",
    "연령대_10_유동인구_수", "연령대_20_유동인구_수", "연령대_30_유동인구_수", 
    "연령대_40_유동인구_수", "연령대_50_유동인구_수", "연령대_60_이상_유동인구_수", 
    "시간대_00_06_유동인구_수", "시간대_06_11_유동인구_수",
    "시간대_11_14_유동인구_수", "시간대_14_17_유동인구_수",
    "시간대_17_21_유동인구_수", "시간대_21_24_유동인구_수",
    "월요일_유동인구_수", "화요일_유동인구_수", "수요일_유동인구_수", "목요일_유동인구_수", "금요일_유동인구_수",
    "토요일_유동인구_수", "일요일_유동인구_수"
]

seoul_people_street_copy = seoul_people_street_copy.drop(columns=columns_to_drop)
seoul_people_street_copy.info()

In [ ]:
seoul_people_street_copy.head()

In [ ]:
seoul_people_street_copy = seoul_people_street_copy.reset_index(drop=True)

In [ ]:
seoul_sales_people_street = pd.merge(seoul_sales_copy_grouped_sort, seoul_people_street_copy, on=["기준_년분기_코드", "상권_구분_코드_명", "상권_코드_명"], how="inner")
seoul_sales_people_street.head()

In [ ]:
seoul_sales_people_street_sorted = seoul_sales_people_street.sort_values(by="기준_년분기_코드", ascending=True)

In [ ]:
seoul_people_street_copy = seoul_people_street_copy.reset_index(drop=True)
seoul_people_street_copy.head()

In [ ]:
seoul_sales_people_street.info()

## 직장인구

In [ ]:
seoul_people_working=pd.read_csv("../data/서울시 상권분석서비스(직장인구-상권).csv", encoding="cp949")
seoul_people_working.shape

In [ ]:
seoul_people_working.head()

In [ ]:
seoul_people_working_copy=seoul_people_working.copy()

In [ ]:
seoul_people_working_copy.info()

In [ ]:
columns_to_drop = [
    "상권_구분_코드", "상권_코드",
    "연령대_10_직장_인구_수", "연령대_20_직장_인구_수", "연령대_30_직장_인구_수", 
    "연령대_40_직장_인구_수", "연령대_50_직장_인구_수", "연령대_60_이상_직장_인구_수", 
    "남성연령대_10_직장_인구_수", "남성연령대_20_직장_인구_수", "남성연령대_30_직장_인구_수",
    "남성연령대_40_직장_인구_수", "남성연령대_50_직장_인구_수", "남성연령대_60_이상_직장_인구_수",
    "여성연령대_10_직장_인구_수", "여성연령대_20_직장_인구_수", "여성연령대_30_직장_인구_수",
    "여성연령대_40_직장_인구_수", "여성연령대_50_직장_인구_수", "여성연령대_60_이상_직장_인구_수"
]

seoul_people_working_copy = seoul_people_working_copy.drop(columns=columns_to_drop)
seoul_people_working_copy.info()

In [ ]:
seoul_people_working_copy.head()

In [ ]:
seoul_sales_people_street_working = pd.merge(seoul_sales_people_street, seoul_people_working_copy, on=["기준_년분기_코드", "상권_구분_코드_명", "상권_코드_명"], how="inner")
seoul_sales_people_street_working.head()

In [ ]:
seoul_sales_people_street_working.rename(columns={"당월_매출_금액": "매출_금액"}, inplace=True)

def seasons(row):
    if row == 20241:
        return "봄"
    elif row == 20242:
        return "여름"
    elif row == 20243:
        return "가을"
    else:
        return "겨울"

seoul_sales_people_street_working["계절"] = seoul_sales_people_street_working["기준_년분기_코드"
                                                                            ].apply(seasons)
seoul_sales_people_street_working.insert(seoul_sales_people_street_working.columns.get_loc("기준_년분기_코드") + 1, "계절", seoul_sales_people_street_working.pop("계절"))

In [ ]:
seoul_sales_people_street_working = seoul_sales_people_street_working.sort_values(by="기준_년분기_코드", ascending=True)

In [ ]:
seoul_sales_people_street_working = seoul_sales_people_street_working.reset_index(drop=True)
seoul_sales_people_street_working.head()

In [ ]:
seoul_sales_people_street_working.info()

In [ ]:
seoul_sales_people_street_working.to_csv("../data/seoul_sales.csv", index=False)

## 데이터 전처리
> 상관관계 확인  
> 각 변수 시각화  

In [ ]:
seoul_data=pd.read_csv("../data/seoul_sales.csv")
seoul_data.info()

### 상관관계 확인

In [ ]:
numeric_columns = seoul_data.select_dtypes(include=["float64", "int64"])
correlation_matrix = numeric_columns.corr()
correlation_matrix

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True)
plt.show()

매출금액을 기준으로 상관성을 보면 다음과 같다.  

In [ ]:
correlation_with_sales = numeric_columns.corr()["매출_금액"].sort_values(ascending=False)
correlation_with_sales

### 시각화

#### 금액관련 변수와 총 매출 금액

In [ ]:
list_sales = ["주중_매출_금액", "주말_매출_금액",
              "남성_매출_금액", "여성_매출_금액",
              "출근_매출_금액", "점심_매출_금액", "퇴근_매출_금액"]

n_cols = 4
n_rows = (len(list_sales) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5 * n_rows))
axes = axes.flatten()

for i, sale in enumerate(list_sales):
    sns.scatterplot(x=numeric_columns[sale], y=numeric_columns["매출_금액"], ax=axes[i])
    axes[i].set_title(f"{sale}과 매출_금액")
    axes[i].set_xlabel(sale)
    axes[i].set_ylabel("매출_금액")
    
for j in range(i + 1, len(axes)):
    axes[j].axis("off")

plt.show()

#### 직장 인구수와 매출 금액 관계

In [ ]:
list_population = ["총_직장_인구_수", "여성_직장_인구_수", "남성_직장_인구_수"]

n_cols = 3
n_rows = (len(list_population) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 5 * n_rows))
axes = axes.flatten()

for i, population in enumerate(list_population):
    sns.scatterplot(x=numeric_columns[population], y=numeric_columns["매출_금액"], ax=axes[i])
    axes[i].set_title(f"{population}와 매출_금액")
    axes[i].set_xlabel(population)
    axes[i].set_ylabel("매출_금액")
    
for j in range(i + 1, len(axes)):
    axes[j].axis("off")

plt.tight_layout()
plt.show()

#### 유동 인구수와 매출 금액 관계

In [ ]:
list_population = ["총_유동인구_수", "여성_유동인구_수", "남성_유동인구_수"]

n_cols = 3
n_rows = (len(list_population) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 5 * n_rows))
axes = axes.flatten()

for i, population in enumerate(list_population):
    sns.scatterplot(x=numeric_columns[population], y=numeric_columns["매출_금액"], ax=axes[i])
    axes[i].set_title(f"{population}와 매출_금액")
    axes[i].set_xlabel(population)
    axes[i].set_ylabel("매출_금액")
    
for j in range(i + 1, len(axes)):
    axes[j].axis("off")

plt.tight_layout()
plt.show()

#### 상권구분별 매출 금액
> 어떤 상권에서의 매출 금액이 가장 높을지  
> 어떤 상권에서 소비가 많이 일어나는지(분포)  

발달상권에서의 판매가 많이 일어나지만, 관광특구에서의 매출 금액이 더 큼

In [ ]:
seoul_data["상권_구분_코드_명"].unique()

In [ ]:
count_by_region = seoul_data.groupby('상권_구분_코드_명').size()
count_by_region

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=seoul_data, x="상권_구분_코드_명", y="매출_금액")
plt.title("상권 구분별 매출 금액")
plt.xlabel("상권 구분")
plt.ylabel("매출 금액")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=seoul_data, x="상권_구분_코드_명", y="매출_금액")
plt.title("상권 구분별 매출 금액")
plt.xlabel("상권 구분")
plt.ylabel("매출 금액")
plt.show()

barplot을보니 관광특구가 너무 높게 나왔다.  
이 이유는 다음과 같다.  

In [ ]:
business_district=seoul_data[seoul_data["상권_구분_코드_명"]=="발달상권"]
business_district["매출_금액"].describe()

In [ ]:
tourist_region=seoul_data[seoul_data["상권_구분_코드_명"]=="관광특구"]
tourist_region["매출_금액"].describe()

먼저 발달상권의 표준편차를 보면 매출의 변동이 상당히 크다.  
그리고 중앙값이 평균보다 낮은 것을 보니 매출이 고르게 분포하지 않고 일부 상권에서 높은 매출이 기록됐음을 알 수 있다.  

관광특구의 경우는 표준편차를 보니 발달상권보다 표준편차가 상대적으로 큰데,  
데이터의 개수가 적기 때문에 변동성이 상대적으로 커보인다.  

즉 발달상권은 매출 차이가 매우 커서 평균 매출이 높아지고, 중앙값은 그보다 낮아서 극단적으로 높은 값들이 평균에 영향을 미쳤다.  
관광특구의 경우는 매출이 상대적으로 고르게 분포하며, 평균이 발달상권보다 높다.  
또한 매출의 차이가 고르게있다.  

이에 따라서 평균으로 나오는 barplot에서 관광특구가 훨씬 높게 나온 것이다.  

#### 결측치 확인

In [ ]:
seoul_data.isna().sum()

#### 이상치 확인 - 기술통계량

In [ ]:
seoul_data.describe().T

상권별로 매출이 다를 수밖에 없기 때문에 이상치를 두고 진행한다.  

## CDA

상권별로 매출 금액의 차이가 있는가?
계절별로 매출 금액의 차이가 있는가?


In [ ]:
sns.histplot(data=tips, x="total_bill", kde=True)

In [ ]:
sns.histplot(data=seoul_data, x="매출_금액", kde=True)

In [ ]:
stats.normaltest(seoul_data["매출_금액"])

In [ ]:
seoul_data["상권_구분_코드_명"].value_counts()

In [ ]:
golmok=seoul_data[seoul_data["상권_구분_코드_명"]=="골목상권"]
jeontong=seoul_data[seoul_data["상권_구분_코드_명"]=="전통시장"]
baldal=seoul_data[seoul_data["상권_구분_코드_명"]=="발달상권"]
gwangwang=seoul_data[seoul_data["상권_구분_코드_명"]=="관광특구"]

In [ ]:
stats.kruskal(golmok["매출_금액"],jeontong["매출_금액"],baldal["매출_금액"],gwangwang["매출_금액"])

In [ ]:
stats.levene(tips_m["tip"], tips_f["tip"])